In [1]:
# insert aima-python to python list of libraries
# needed to import aima-python
import sys
import random
import matplotlib.pyplot as plt
PATH_TO_AIMA = "/home/andreza/Documents/1S2020/mc906/aima-python"
if PATH_TO_AIMA not in sys.path:
    sys.path.insert(0,PATH_TO_AIMA)
print(sys.path)

['/home/andreza/Documents/1S2020/mc906/aima-python', '/home/andreza/Documents/1S2020/mc906/ai-pacman', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/andreza/.local/lib/python3.8/site-packages', '/usr/lib/python3.8/site-packages', '/home/andreza/.local/lib/python3.8/site-packages/IPython/extensions', '/home/andreza/.ipython']


In [2]:
# import testing
from search import *
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens
import walls

In [3]:

class CellType:
    WALL_GHOST = 1
    DOT = 2
    PACMAN = 3
    EMPTY = 0

class Maze:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def __getitem__(self, position):
#         x, y = position
        return self.maze[position]
    
    def __setitem__(self, new_value, position):
        self.maze[position] = new_value
    
    def __hash__(self):
        return id(self.maze)
    
    def __eq__(self, other):
        return (self.maze == other).all()
        
    def show_maze(self):
        plt.imshow(self.maze, interpolation='nearest')
        plt.show()
              
    def maze_to_array(self, maze_str, width, height):
        """
        Function that converts string maze to numpy array
        """
        maze = []
        for line in str(maze_str).splitlines():
            line = line+line[::-1]
            if line:
                line = list(line)
                for item in range(len(line)):
                    if line[item] == "|":
                        line[item] = CellType.WALL_GHOST
                    else:
                        line[item] = CellType.DOT

                maze.append(line)
        return np.array(maze)         
    
    def maze_generator(self):
        """
        Function that creates the base string to generate maze
        """
        maze = np.zeros((self.height, self.width))
        maze[0,:] = 1
        maze[:,0] = 1
        maze[-1,:] = 1
        maze[self.height//2-2:self.height//2+2, self.width-3::] = 1
        maze_str = []
        for line in maze:
            for item in line:
                if item == 1:
                    maze_str.append('|')
                else:
                    maze_str.append('.')
            maze_str.append('\n')
        maze_str = ''.join(maze_str)

        maze = walls.Map(self.height, self.width, maze_str)
        # verbosity option (-v)
        if len(sys.argv) > 1 and sys.argv[1] == "-v":
            maze.verbose = True
        # generate map by adding walls until there's no more room
        while maze.add_wall_obstacle(extend=True):
            pass
        
        maze_array = self.maze_to_array(maze, self.width, self.height)
        
        # insert ghosts to the maze
        for i in range(3):
            valid_positions = np.argwhere(maze_array==CellType.DOT)
            position = random.choice(valid_positions)
            maze_array[position[0],position[1]] = CellType.WALL_GHOST
        
        # pacman initial position
        valid_positions = np.argwhere(maze_array==CellType.DOT)
        pac_position = random.choice(valid_positions)
        maze_array[pac_position[0],pac_position[1]] = CellType.PACMAN
        self.maze = maze_array
        self.maze.flags.writeable = False


    # maybe get shape

In [4]:
class PacmanMazeProblem(Problem):
    """The Pacman Maze Problem."""
    
    
    def __init__(self, maze, goal_position):
        Problem.__init__(self, maze, goal_position)
        
    def actions(self, state):
        """The possible actions are the neighbors of a given node
        except if the neighbor is a ghost or a wall"""
        
        actions = []
        y, x = np.where(state.maze == CellType.PACMAN)
        y = y[0]
        x = x[0]
        directions = [         (0, -1),
                      (-1, 0),           (1,  0),
                                (0, 1)           ]
        for position in directions:
            if (y+position[1]) >=0 and (y+position[1]) < state.height:
                if (x+position[0]) >=0 and (x+position[0]) < state.width:
                    possible_pos = (y+position[1], x+position[0])
                    maze_value = state.maze[possible_pos]
                    if maze_value != CellType.WALL_GHOST:
                        actions.append(possible_pos)
        return actions

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state."""
        
        new_state = Maze(state.width, state.height)
        new_state.maze = state.maze.copy() 
        
        y, x = np.where(new_state.maze == CellType.PACMAN)
        y = y[0]
        x = x[0]

        new_state.maze[y][x] = CellType.EMPTY
        y = action[0]
        x = action[1]
        new_state.maze[y][x] = CellType.PACMAN
        
        return new_state
    
    def goal_test(self, state):
        """Return True if the state is a goal."""
        y, x = np.where(state.maze == CellType.PACMAN)
        y = y[0]
        x = x[0]
        
        return y == self.goal[0] and x == self.goal[1]

    def path_cost(self, c, state1, action, state2):
        """
        If action makes pacman eat a DOT this path is less costly
        """
        if state1.maze[action] == CellType.DOT:
            return c + 1
        else:
            return c + 2
    
#     def h1():  

In [5]:
MAZE_WIDTH = 10
MAZE_HEIGHT = 27
maze_array = Maze(MAZE_WIDTH, MAZE_HEIGHT)
maze_array.maze_generator()


In [6]:
# goal position
valid_positions = np.argwhere(maze_array.maze==CellType.DOT)
goal_position = random.choice(valid_positions)

In [7]:
problem = PacmanMazeProblem(maze_array, goal_position)

In [11]:
print(goal_position)
print(np.where(maze_array.maze == CellType.PACMAN))
solution = breadth_first_graph_search(problem).solution()
# tchururuuuuu

[ 2 24]
(array([0]), array([17]))


AttributeError: 'NoneType' object has no attribute 'solution'

In [9]:
print(solution.solution())
print(solution.path_cost)

AttributeError: 'NoneType' object has no attribute 'solution'

In [ ]:
print(np.where(maze_array.maze == CellType.PACMAN))
print(goal_position)
maze_array.show_maze()